### CTF ShittyAddon

To be run on a Raspberry Pi, which talks to the SAO via `I2C`

#### General

- Flag is kept in flash, not in SRAM

#### Understanding what the DUTs firmware does

- `setup()`
    - Sets up own address to be `0x23`
    - Hooks up I2C interrupts
    - XORs `flash_value` (Initially `0x20`) with each char in the flag individually
- `loop()`
    - `erase` gets HIGH when `flash_addr` has `0x8000` set
    - `flash_addr` gets ANDed with `0x1fff`
    - Whenever `flash_addr` > `0x800`:
        - If `erase` is HIGH:
            - Call `boot_page_erase_safe`
        - else
            - Call `boot_page_fill_safe`, writing out `flash_value` to `flash_addr`
            - Call `boot_page_write` on `flash_addr`
    - Reset `flash_addr` to 0
- `onI2CReceive()`
    - Writes first byte received into `target`
    - Uses `target` as a pointer and writes subsequently received bytes into the target location
- `onI2CRequest()`
    - Sends the current value of `target`
    - Interprets `target++` as a pointer, limiting it to `0x1fff`, putting it into `addr`
    - Seems to send a byte found at an adress, but needs to decide if it can read from pgm or directly?
        - If `target & 0x8000`
            - Sends a byte read from the program at adress `addr`
        - else
            - Sends the dereferenced value of `addr`


##### Flag XOR thingy

In [ ]:
# Starting values
flash_value = 0x20
flag = "$FLAG:SECRET"

# XOR flash_value with flag like setup()
for char in flag:
    flash_value ^= ord(char)
    print(char + " = " + str(ord(char)) + " | flash_value --> " + str(flash_value))


##### `flash_addr` simulation

The whole loop seems to be used only for the rootkit stuff. There seems to be a limiter to `flash_addr`, preventing write access above `0x1fff`, as well as methods to delete and write stuff into the boot page

In [ ]:
# Declare start value
flash_addr  = 0x400
erase       = False

print(0x1fff) # Limits flash_addr
print(0x8000) # Sets "erase"
print(0x0800) # Values above start executing stuff below

# Simulate a few iterations
for i in range (10):
    erase = flash_addr & 0x8000
    flash_addr &= 0x1fff
    print("Erase = " + str(erase) + " | flash_addr = " + hex(flash_addr))

#### Test if we can talk to DUT

In [240]:
from smbus2 import SMBus

dutAdr      = 0x23          # Address of the DUT
adrs = [0x18, 0x19, 0x1A, 0x1B, 0x1C, 0x3B]

with SMBus(1) as bus:
    
    for adr in reversed(adrs):
        
        #bus.write_byte(dutAdr, adr)
        response = bus.read_i2c_block_data(dutAdr, adr, 2)

        print(hex(response[0]) + ' | ' + bin(response[1]))

0x1d | 0b0
0x1c | 0b11100
0x1b | 0b0
0x1a | 0b11000101
0x19 | 0b0
0x18 | 0b11001


#### Attack 1 - Make LED light up

In [201]:
import smbus2

channel     = 1             # Used I2C channel
dutAdr      = 0x23          # Address of the DUT
portAdr     = 0x18          # Address of PORTB, where the LED is connected to
ddrAdr      = 0x17          # Address of DDRB, to set the LED pin as an output
ledBit      = 0b00000010    # Bit in PORTB where the LED is connected to

# Init I2C
bus = smbus2.SMBus(channel) 

###############
# Update DDRB
###############
print('Reading DDRB')

# Write target address into DUT
#bus.write_byte(dutAdr, ddrAdr)

# Read byte at target location from DUT
stateDDRB   = bus.read_i2c_block_data(dutAdr, ddrAdr, 2)
print('DDRB Adr = '   + hex(stateDDRB[0]))
print('Old DDRB = ' + bin(stateDDRB[1]))

# Set LED pin to output
stateDDRB[1] |= ledBit

print('New DDRB = ' + bin(stateDDRB[1]))

# Write DDRB back into DUT
data = [ddrAdr, stateDDRB[1]]
#print(hex(data[0]))
bus.write_i2c_block_data(dutAdr, 0, data)

stateDDRB   = bus.read_i2c_block_data(dutAdr, ddrAdr, 2)[1]
print('Confirmed DDRB = ' + bin(stateDDRB))

Reading DDRB
DDRB Adr = 0x17
Old DDRB = 0b11101100
New DDRB = 0b11101110


TimeoutError: [Errno 110] Connection timed out